In [71]:
import requests
import json
import pandas as pd
import os
import numpy as np
import hvplot.pandas
from utils.utils import coin_gecko_cleaner
%matplotlib inline

In [72]:
# Pulling in api data for USDC
USDC_url = 'https://api.coingecko.com/api/v3/coins/usd-coin/market_chart?vs_currency=usd&days=1095&interval=daily'

In [73]:
# Converting api to json format
USDC = requests.get(USDC_url).json()

In [74]:
# Converting json into DataFrame
USDC_df=pd.DataFrame(USDC)

In [75]:
display(USDC_df)

,prices,market_caps,total_volumes
0,"[1560384000000, 0.9987240389027506]","[1560384000000, 341070351.6987576]","[1560384000000, 2049717159.0398748]"
1,"[1560470400000, 1.000988185025491]","[1560470400000, 344673326.6087941]","[1560470400000, 2588629188.4414473]"
2,"[1560556800000, 0.9961121599121703]","[1560556800000, 348823456.55398226]","[1560556800000, 1883465392.2007582]"
3,"[1560643200000, 0.9983310853331963]","[1560643200000, 349968804.5407158]","[1560643200000, 2699800279.826856]"
4,"[1560729600000, 1.001112050231655]","[1560729600000, 349808117.16019255]","[1560729600000, 2797432348.6743093]"
...,...,...,...
1090,"[1654560000000, 1.001435458436979]","[1654560000000, 53828413758.921135]","[1654560000000, 4261652446.8086476]"
1091,"[1654646400000, 1.0020084453743354]","[1654646400000, 53906740225.190636]","[1654646400000, 5791456663.372619]"
1092,"[1654732800000, 1.00070856363376]","[1654732800000, 53755487848.0945]","[1654732800000, 4154645563.673361]"
1093,"[1654819200000, 1.0008539882774279]","[1654819200000, 53849615435.72309]","[1654819200000, 3005606416.353044]"


In [76]:
# The data is returned with epoch time listed as part of the array for every list item. We want to set our index as epoch time, and then change that to DateTime for clarity. 
# The Epoch time also had extra zeros included at the end, which skewed it when converted to regular Datetime. 
# We will address those issues with the following functions:
def extract_epoch_time(item):
    # item is expected to be an array where the 1st element in the array is the epoch time. 
    # this function returns the epoch time.
    return item[0]

def extract_value(item):
    # item is expected to be an array where the 2nd element in the array is the value. 
    # this function returns the value.
    return item[1]

In [77]:
# cleaning url
coin_gecko_cleaner(USDC_url)

,prices,market_caps,total_volumes
time,,,
2019-06-12,0.998724,3.410704e+08,2.049717e+09
2019-06-13,1.000988,3.446733e+08,2.588629e+09
2019-06-14,0.996112,3.488235e+08,1.883465e+09
2019-06-15,0.998331,3.499688e+08,2.699800e+09
2019-06-16,1.001112,3.498081e+08,2.797432e+09
...,...,...,...
2022-06-06,1.001435,5.382841e+10,4.261652e+09
2022-06-07,1.002008,5.390674e+10,5.791457e+09
2022-06-08,1.000709,5.375549e+10,4.154646e+09


In [78]:
USDC = coin_gecko_cleaner(USDC_url)
display(USDC)
type(USDC['prices'])

,prices,market_caps,total_volumes
time,,,
2019-06-12,0.998724,3.410704e+08,2.049717e+09
2019-06-13,1.000988,3.446733e+08,2.588629e+09
2019-06-14,0.996112,3.488235e+08,1.883465e+09
2019-06-15,0.998331,3.499688e+08,2.699800e+09
2019-06-16,1.001112,3.498081e+08,2.797432e+09
...,...,...,...
2022-06-06,1.001435,5.382841e+10,4.261652e+09
2022-06-07,1.002008,5.390674e+10,5.791457e+09
2022-06-08,1.000709,5.375549e+10,4.154646e+09


pandas.core.series.Series

In [79]:
## Define Desired Deviation

deviation_high = 1.001
deviation_low = .999

In [80]:
## List Initiation
current_price = USDC['prices'].iloc[0]
print(current_price)
type(current_price)

0.9987240389027506


numpy.float64

In [81]:
trigger_counter = 0
for price_to_check in USDC['prices']:
    max_deviation_low = deviation_low * current_price 
    max_deviation_high = deviation_high * current_price 
    
    if price_to_check < max_deviation_low:
        print("Found a LOW deviation. Price update required")
        print("Price being compared: " + str(price_to_check))
        print("Max low deviation: " + str(max_deviation_low))
        print("Setting new \"current price\" to: " + str(price_to_check))
        current_price = price_to_check # take the price we just looked at and set it as the new baseline for deviation calculations
        trigger_counter = trigger_counter + 1 # add one to our count of times triggered
        print("Times trigger has been... triggered: " + str(trigger_counter) + "\n")
        
    if price_to_check > max_deviation_high:
        print("Found a HIGH deviation. Price update required")
        print("Price being compared: " + str(price_to_check))
        print("Max high deviation: " + str(max_deviation_high))
        print("Setting new \"current price\" to: " + str(price_to_check))
        current_price = price_to_check # take the price we just looked at and set it as the new baseline for deviation calculations
        trigger_counter = trigger_counter + 1 # add one to our count of times triggered 
        print("Times trigger has been... triggered: " + str(trigger_counter) + "\n")

        

Found a HIGH deviation. Price update required
Price being compared: 1.000988185025491
Max high deviation: 0.9997227629416533
Setting new "current price" to: 1.000988185025491
Times trigger has been... triggered: 1

Found a LOW deviation. Price update required
Price being compared: 0.9961121599121703
Max low deviation: 0.9999871968404654
Setting new "current price" to: 0.9961121599121703
Times trigger has been... triggered: 2

Found a HIGH deviation. Price update required
Price being compared: 0.9983310853331963
Max high deviation: 0.9971082720720823
Setting new "current price" to: 0.9983310853331963
Times trigger has been... triggered: 3

Found a HIGH deviation. Price update required
Price being compared: 1.001112050231655
Max high deviation: 0.9993294164185293
Setting new "current price" to: 1.001112050231655
Times trigger has been... triggered: 4

Found a LOW deviation. Price update required
Price being compared: 0.9993317005591263
Max low deviation: 1.0001109381814235
Setting new "c